# Partie 1 : Démarrage

## 1.1

In [1]:
import pandas as pd
import csv

df = pd.read_csv(
    "discours_US.csv",
    sep="\t",              
    quoting=csv.QUOTE_NONE  
)

print(df.shape)
print(df.columns)

import numpy as np

df.columns = df.columns.str.replace('"', '', regex=False).str.strip()
df = df.replace('"', '', regex=True)
print(df.head())

(164, 5)
Index(['"speaker"', '"text"', '"date"', '"descr"', '"link"'], dtype='object')
   speaker                                               text            date  \
0  CLINTON  : I'm getting ready for a lot of things, a lot...  April 12, 2015   
1  CLINTON  [ ] : I'll be graduating in May, and on gradua...  April 14, 2015   
2  CLINTON  : Well, thank you all so much for inviting me ...  April 20, 2015   
3  CLINTON  Thank you so much. I am absolutely delighted t...  April 29, 2015   
4  CLINTON  Oh, hello. Hi, how are you? Well, it's wonderf...     May 5, 2015   

                                               descr  \
0   Video Remarks Announcing Candidacy for President   
1  Remarks in a Question and Answer Session at Ki...   
2                    Remarks in Keene, New Hampshire   
3  Address to the David N. Dinkins Leadership & P...   
4  Remarks at a Roundtable with Young Nevada Resi...   

                                                link  
0  http://www.presidency.ucsb.edu/

## 1.2

In [2]:
print(df['speaker'].value_counts())

speaker
CLINTON    93
TRUMP      71
Name: count, dtype: int64


## 1.3

In [3]:
from Corpus import Corpus
from Document import Document
import re

corpus_us = Corpus("discours_US")

for _, row in df.iterrows():
    auteur = str(row['speaker']).strip()
    texte  = str(row['text']).strip()
    date   = str(row['date']).strip()
    url    = str(row['link']).strip()
    titre  = str(row['descr']).strip()

    if not texte:
        continue

    for phrase in re.split(r"[.!?]", texte):
        phrase = phrase.strip()
        if phrase:
            doc = Document(titre, auteur, date, url, phrase)
            corpus_us.add_doc(doc)

corpus_us   


Corpus 'discours_US' : 33729 documents, 2 auteurs

## 1.4

In [4]:
corpus_us.search("US")

corpus_us.concorde("president", 30)        


Passages avec le mot: 'US'
So you can do more than just get by, you can get ahead and stay ahead, because when families are strong, America is strong
So I'm hitting the road to earn your vote, because it's your time and I hope you'll join me on this journey
I just had a terrific tour of the advanced manufacturing laboratory
This is a real model for how we can provide more continuous learning starting in high school but going as far as necessary to help better prepare not only young people but all people for the economy that is awaiting us
And before we get started — because I want to hear from each of you, I want more information about what you see as challenges as students and educators and the opportunities you hope to take advantage of — I just want to tell you a little bit about why I'm here today
And our economy and our country are much better off because American families have basically done whatever it took to make it work
There's something wrong when American workers keep getti

,Document,Contexte gauche,Motif trouvé,Contexte droit
0,Video Remarks Announcing Candidacy for President,I'm running for,president,
1,Remarks in a Question and Answer Session at Ki...,And I'm running for,president,because I think that American
2,"Remarks in Keene, New Hampshire",uld love to represent as your,president,
3,"Remarks in Keene, New Hampshire",up — I really approve of the,president,'s proposal to try to make com
4,"Remarks in Keene, New Hampshire","pective, I would want to be a",president,who made it as easy as possib
...,...,...,...,...
1157,Remarks in New York City Conceding the 2016 Pr...,would have made history as a,president,"in one sense, but we never ha"
1158,Remarks in New York City Conceding the 2016 Pr...,ve had a,president,who's made their whole career
1159,Remarks in New York City Conceding the 2016 Pr...,ke history as the first woman,president,
1160,Remarks in New York City Conceding the 2016 Pr...,that he will be a successful,president,for all Americans


# Partie 2 : Utilisation de votre moteur de recherche

## 2.1 & 2.2 & 2.3

In [5]:
from importlib import reload
import SearchEngine
reload(SearchEngine)

from SearchEngine import SearchEngine

sengine = SearchEngine(corpus_us)
sengine.search("ruth",10)        


Construction des résultats:   0%|          | 0/10 [00:00<?, ?it/s]

,score,titre,auteur,date,url,texte
0,0.447768,"Remarks at Cowles Commons in Des Moines, Iowa",CLINTON,"September 29, 2016",http://www.presidency.ucsb.edu/ws/index.php?pi...,"And I saw my good friend Ruth Harkin, and she ..."


# Partie 3 : Petite interface

## 3.1

In [6]:
import ipywidgets as widgets
from IPython.display import display

titre = widgets.Label(value="Moteur de recherche")

requete = widgets.Text(
    description="Mots-clés :",
    placeholder="Ecrivez vos mots-clés ici",
)

slider = widgets.IntSlider(
    description="Nb docs :",
    min=1,
    max=50,
    value=10,
)


display(titre, requete, slider)


Label(value='Moteur de recherche')

Text(value='', description='Mots-clés :', placeholder='Ecrivez vos mots-clés ici')

IntSlider(value=10, description='Nb docs :', max=50, min=1)

## 3.2 à 3.7

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import datetime

df["date"] = pd.to_datetime(df["date"])

titre = widgets.Label(value="Moteur de recherche sur les discours US")

requete = widgets.Text(
    description="Mots-clés :",
    placeholder="Ecrivez vos mots-clés ici",
)

slider = widgets.IntSlider(
    description="Nb docs :",
    layout=widgets.Layout(width="500px"),
    min=1,
    max=50,
    value=10,

)

output = widgets.Output()

auteurs = sorted(df["speaker"].dropna().unique())
filtre_ateur = widgets.Dropdown(
    options=["(Tous)"] + auteurs,
    description="Auteur :",
)

dates_uniques = sorted(df["date"].dt.date.unique())
options_dates = [(d.strftime("%Y-%m-%d"), d) for d in dates_uniques]

barre_dates = widgets.SelectionRangeSlider(
    options=options_dates,
    index=(0, len(options_dates) - 1),
    description="Dates :",
    layout=widgets.Layout(width="500px"),

)

bouton_recherche = widgets.Button(
    description="Rechercher",
)

def clique_bouton(b):
    with output:
        output.clear_output()
        req = requete.value          
        article = slider.value

        df_resultats = sengine.search(req, n=article)
        if filtre_ateur.value != "(Tous)":
            df_resultats = df_resultats[df_resultats["auteur"] == filtre_ateur.value]

        df_resultats["date"] = pd.to_datetime(df_resultats["date"])

        date_debut, date_fin = barre_dates.value
        df_resultats = df_resultats[
            (df_resultats["date"] >= pd.to_datetime(date_debut)) &
            (df_resultats["date"] <= pd.to_datetime(date_fin))
        ]
        date_debut, date_fin = barre_dates.value
       
        display(df_resultats)

bouton_recherche.on_click(clique_bouton)

haut = widgets.HBox([requete, filtre_ateur, bouton_recherche])
filtres = widgets.HBox([slider, barre_dates])
ui = widgets.VBox([titre, haut, filtres, output])

display(ui)
